In [3]:
import pandas as pd
import numpy as np

In [4]:
df = pd.read_pickle("/kaggle/input/data0-25/data_norm0.25.pkl")

In [5]:
df

,OUTPUT,INPUT
0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.1242904446...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ..."
2,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.0770836309982...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,"[0, 0, 0, 0, 0, 0.10748772366243657, 0, 0, 0.0...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,"[0, 0, 0, 0, 0, 0, 0, 0, 0.06577925136276556, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
...,...,...
299995,"[0, 0.048603088169820896, 0, 0, 0.075507061551...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
299996,"[0, 0, 0, 0, 0, 0, 0.4080602882460878, 0, 0, 0...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
299997,"[0, 0.05665944660435852, 0, 0, 0.0880229730938...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
299998,"[0, 0, 0.12884629874068773, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [ ]:
print(len(df["INPUT"][0]))
print(len(df["OUTPUT"][0]))

989

49


In [4]:
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from warnings import filterwarnings
import seaborn as sns
import cv2
from keras.models import Sequential
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
import os
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, TensorBoard, ModelCheckpoint

In [30]:
from torch.utils.data import Dataset
from torch.utils.data import DataLoader


class CustomDataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        x = torch.tensor(self.data.iloc[idx, 0], dtype=torch.float)
        y = torch.tensor(self.data.iloc[idx, 1], dtype=torch.float)
        return x, y

dataset = CustomDataset(df)
dataloader = DataLoader(dataset, batch_size=32, shuffle=False)

In [31]:
import torch
import torch.nn as nn

class CustomModel(nn.Module):
    def __init__(self):
        super(CustomModel, self).__init__()
        self.fc1 = nn.Linear(989, 989)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(989, 49)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.sigmoid(x)
        return x



In [8]:
from tqdm import tqdm

In [32]:
model = CustomModel()
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters())


for epoch in range(10):
    with tqdm(total=len(dataloader), desc=f'Epoch {epoch+1}') as pbar:
        for targets, inputs in dataloader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()
            pbar.update(1)
            pbar.set_postfix({'loss': loss.item()})

    print(f'Epoch {epoch+1}, Loss: {loss.item()}')



Epoch 1: 100%|██████████| 9375/9375 [04:05<00:00, 38.25it/s, loss=0.0501]


Epoch 1, Loss: 0.05007713660597801


Epoch 2: 100%|██████████| 9375/9375 [04:19<00:00, 36.08it/s, loss=0.0489]


Epoch 2, Loss: 0.04891650378704071


Epoch 3: 100%|██████████| 9375/9375 [04:21<00:00, 35.81it/s, loss=0.0484]


...

Epoch 8, Loss: 0.04739661142230034


Epoch 9: 100%|██████████| 9375/9375 [05:34<00:00, 28.00it/s, loss=0.0473]


Epoch 9, Loss: 0.04726368933916092


Epoch 10: 100%|██████████| 9375/9375 [05:34<00:00, 28.06it/s, loss=0.0471]

Epoch 10, Loss: 0.04707371070981026


In [36]:
torch.save(model, '/kaggle/working/first_try.pth')

In [57]:
n = 30000
out = model.forward(torch.tensor(df["INPUT"][n])).tolist()
rounded_out = [round(num, 3) for num in out]
for i in range(49):
    print(rounded_out[i], df["OUTPUT"][0][i])

0.0 0
0.0 0
0.0 0
0.0 0
0.0 0
0.0 0
0.0 0
0.163 0
0.0 0
0.0 0
0.0 0
0.0 0.12429044460990353
0.0 0
0.23 0.19171203430383882
0.0 0
0.0 0.04984197229703562
0.0 0
0.0 0
0.0 0
0.0 0
0.0 0
0.0 0
0.0 0
0.0 0.07333003867293564
0.0 0.11057936110639896
0.304 0
0.0 0
0.0 0
0.0 0
0.0 0
0.0 0
0.0 0
0.0 0
0.0 0
0.003 0.17579340398940366
0.0 0
0.0 0
0.0 0
0.0 0
0.0 0
0.124 0
0.0 0
0.0 0
0.0 0
0.0 0.11367177304035844
0.0 0.1607809719801254
0.0 0
0.0 0
0.172 0
